<a href="https://colab.research.google.com/github/GeoKauko/TheNavySeals/blob/main/3_Postprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Postprocessing

In [ ]:
## Import packages
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
## GOOGLE COLAB USERS ONLY 
## Mount Google Drive for data retrieval

from google.colab import drive
drive.mount('/content/drive')

import os

project_path = '/content/drive/My Drive/TheNavySeals/'

In [ ]:
## LOCAL USERS ONLY
## Change the path to your project directory

os.chdir('D:\E_2024_P6\SEAL')

project_path = ''

In [ ]:
## Load functions from preprocessing
%run "1_Preprocessing.ipynb"

In [ ]:
## Define paths
source_image
reduced_image_path = 'data/3_postprocessing/3a_reduced_image'
os.makedirs(reduced_image_path, exist_ok=True)
reduced_image = os.path.join(reduced_image_path, '1a_reduced_image.tif')

tiled_images_path = 'data/3_postprocessing/3b_tiled_images'
os.makedirs(tiled_images_path, exist_ok=True)

predicted_masks_path = 'data/3_postprocessing/3c_predicted_masks'
os.makedirs(predicted_masks_path, exist_ok=True)

predicted_mask_final = 'data/3_postprocessing/3d_predicted_mask_final'
os.makedirs(predicted_mask_final, exist_ok=True)
predicted_mask = os.path.join(predicted_masks_path, '3d_predicted_mask.tif')

mask_heatmap_path = 'data/3_postprocessing/3e_mask_heatmap'
os.makedirs(predicted_mask_final, exist_ok=True)
mask_heatmap = os.path.join(predicted_masks_path, '3e_mask_heatmap.tif')

In [ ]:
## Process the image
# Reduce image for increased processing speed
reduce_radiometric_resolution(source_image, reduced_image)
# Tile the image into 224x224 px tiles
split_and_save_raster(reduced_image, 224, 224, tiled_images_path)

In [ ]:
def is_multiband_pil(image_path):
    """
    Check if an image is multiband or singleband using Pillow.
    
    Args:
        image_path (str): Path to the image file.
    
    Returns:
        bool: True if the image is multiband (color), False if it is singleband (grayscale).
    """
    try:
        with Image.open(image_path) as img:
            return img.mode in ("RGB", "RGBA", "CMYK", "YCbCr")
    except Exception as e:
        print(f"Error opening image: {e}")
        return False

In [ ]:
## Process predicted masks into a heatmap
# Mosaic the masks back together
mosaic_rasters(predicted_masks_path, predicted_mask_final)

# Create a heatmap
def create_heatmap(input_path, output_path):
    """
    Save a heatmap from the given data to a file.
    
    Args:
        input_path (str): Path to the mask raster.
        output_path (str): Path to save the heatmap image.
    """
    with rasterio.open(input_path) as src:
        input_data = src.read(1)
    plt.figure(figsize=(10, 8))
    plt.imshow(input_data, cmap='hot', interpolation='nearest')
    plt.colorbar(label='Value')
    plt.title('Heatmap of mask.tif')
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    
    plt.savefig(output_path, bbox_inches='tight')
    plt.close()

create_heatmap(predicted_mask_final, mask_heatmap)